In [1]:
import numpy as np;
import time;

In [2]:
from RBComb.SerialBridge import *
from RBComb.RBComb576 import *
from RBComb.Helper.BoardPinout import *
from RBComb.Helper.BoardUnits import *;
from RBComb.Helper.FunctionInterpolator import *

In [3]:
# This document shows how 2 signal generators with positive/negative amplitude can be combined
# to make a single signal generator with arbitrary phase.
# This is done by phase-shifting one of the generators by 90 degrees so it acts as the imaginary
# part of the amplitude.

In [4]:
# Construct bridge:
currentBridge = SerialBridge('/dev/tty.usbserial-FT35HA60B', True, 300);

# Note: the correct baud rate is 5'000'000 but it is not supported in Mac OS X
# The solution is to modify the serial port driver so it aliases ("pretends") to be working at a standard
# rate of 300, but in reality it works at 5000000. This is done by modifying /Library/Extensions/FTDIUSBSerialDriver.kext
# following the guide https://www.ftdichip.com/Support/Documents/TechnicalNotes/TN_105%20Adding%20Support%20for%20New%20FTDI%20Devices%20to%20Mac%20Driver.pdf
# And subsequently disabling the "trusted platform" features of mac since it would no longer accept the modified extension

In [5]:
# Construct RBComb576
rbComb = RBComb576(currentBridge);

In [6]:
# Construct helper objects:
pinout = BoardPinout();
units = BoardUnits();

In [18]:
# We must make sure the clock is disabled, otherwise the boards cannot hear us
# as the bridge does not forward commands when the clock is on:
rbComb.disableClock()

# Select all boards
rbComb.selectAllBoards()

# Ramp up (won't take effect until turning on the clock)
rbComb.outputRampDown();


In [19]:
# We use the interpolator to raise the signal to V/2 as the output cannot take negative values
interpolator = FunctionInterpolator();
constantFunction = interpolator.getConstantFunction(0.5)

# Upload offset function to the device (Takes 10 seconds, do only once per "session")
for i in range(576):
    rbComb.setTaylorCoefficients(i, False, constantFunction);

In [25]:
# Parameters for the sine generators
frequency = 40000.0; # 1 KHz

# Amplitudes and phases are independent for every channel:
amplitudes = np.array([units.convertNormalizedAmplitudeToInteger(0.5)]*576); 
phases = np.array([0]*576); # In degrees.

# We will phase shift the output 2,2 by 45 degrees:
phases[pinout.getOutputNumber(2,2)] = 45;

In [22]:
# Compute real and imaginary amplitudes
amplitude_real =  amplitudes*np.cos(phases*np.pi/180);
amplitude_imag =  amplitudes*np.sin(phases*np.pi/180);

In [32]:
# Configure the two sine generators:

# Ensure that the two signal generators are on. This is the default state at power-on, but may be
# affected by previously run experiments if the board has not been unpowered:
rbComb.setCurrentState([1,1,0,0,0]);

# Configure phases:
rbComb.setPhase(0, units.convertDegreesPhaseToInteger(0))
rbComb.setPhase(1, units.convertDegreesPhaseToInteger(90))

# Configure frequencies:
rbComb.setFrequency(0, units.convertFrequencyHzToInteger(frequency))
rbComb.setFrequency(1, units.convertFrequencyHzToInteger(frequency))

#rbComb.setFrequency(0, 2**29)
#rbComb.setFrequency(1, 2**29)

# Make sure that the generators don't sweep
for i in range(4):
    rbComb.setSweepFrequencyStep(i,0);

# Configure amplitudes: 
for i in range(576):
    rbComb.setAmplitude(0,i,int(amplitude_real[i]));
    rbComb.setAmplitude(1,i,int(amplitude_imag[i]));
    # Switch off the other two generators by setting their amplitudes to zero:
    rbComb.setAmplitude(2,i,0);
    rbComb.setAmplitude(3,i,0);

In [33]:
# Perform experiment:
rbComb.outputRampUp();
rbComb.enableClock();
#time.sleep(6);
#rbComb.disableClock()

In [ ]:
# Ramp down the output
#rbComb.outputRampDown();
#rbComb.enableClock();
#time.sleep(4);
#rbComb.disableClock()

In [31]:
rbComb.disableClock()

In [ ]:
# Close the serial port:
#currentBridge.closeConnection()

In [36]:
r = np.pi*4.9*4.9-(6*4);

In [37]:
r

51.42963961269095